In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import requests
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium

In [2]:
df = pd.read_csv("../data/Geospatial_Coordinates.csv")

In [3]:
df.shape

(103, 3)

In [4]:
import os
api_key = os.environ["FOURSQUARE_API_KEY"]
#YELP_api_key = os.environ["YELP_API_KEY"]

In [166]:
df.Latitude.nunique()

103

In [142]:
def get_POI(latitude,longitude):
    
    url = "https://api.foursquare.com/v3/places/search?fields=name,location,categories,rating,popularity,price"
    
    headers = {"Accept": "application/json"}
    headers['Authorization'] = api_key
    
    location = [latitude,longitude]
    location_str = [str(x) for x in location]
    location_join = ','.join(location_str)
    
    params_dict = {
                   "query" : "Food",
                   "ll" : location_join,
                   #"categories" : 
                   "radius" : "2000",
                   "limit" : "50"
    }
    
    response = requests.get(url, params_dict, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f'Error {response.status_code}')
    

In [114]:
Restaurant_data = df.apply(
                            lambda row: get_POI(row['Latitude'], row['Longitude']),
                            axis=1)

In [155]:
Restaurant_data[0]['results'][1]

{'categories': [{'id': 13145,
   'name': 'Fast Food Restaurant',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
    'suffix': '.png'}}],
 'location': {'address': '8129 Sheppard Ave E',
  'country': 'CA',
  'cross_street': '',
  'formatted_address': '8129 Sheppard Ave E, Scarborough ON M1B 6A3',
  'locality': 'Scarborough',
  'postcode': 'M1B 6A3',
  'region': 'ON'},
 'name': "Wendy's",
 'popularity': 0.9958471041132856,
 'price': 1,
 'rating': 6.2}

In [116]:
fs_rest_api=[]
def restaurant_POI(data):
    for row in data:
        for val in row['results']:
            try:
                fs_rest_api.append(
                                [
                                   val['name'],
                                   val['categories'][0]['name'],
                                   val['location']['postcode'],
                                   val['popularity'],
                                   val['rating'],
                                   val['price']
                                ]
                                    )
            except KeyError:
                continue
    

In [117]:
restaurant_POI(Restaurant_data)

In [130]:
df_restaurants = pd.DataFrame(fs_rest_api, columns= ['name','category', 'Postal_code','popularity', 'rating','price'])

In [139]:
df_restaurants[df_restaurants['name'] == "Bella's Lechon"]

,name,category,Postal_code,popularity,rating,price


In [132]:
df_restaurants['Postal_code'] = df_restaurants['Postal_code'].apply(lambda x: x[:3])

In [140]:
df_restaurants.head()

,name,category,Postal_code,popularity,rating,price
0,Subway,Deli,M1B,0.860107,6.4,1
1,Wendy's,Fast Food Restaurant,M1B,0.995847,6.2,1
2,Harvey's,Fast Food Restaurant,M1B,0.982336,6.3,2
3,Caribbean Wave,American Restaurant,M1B,0.988098,6.4,1
4,Swiss Chalet Rotisserie & Grill,Restaurant,M1B,0.948866,5.2,2


In [143]:
def get_transp_POI(latitude,longitude):
    
    url = "https://api.foursquare.com/v3/places/search?fields=name,location,categories,rating,popularity,price,features"
    
    headers = {"Accept": "application/json"}
    headers['Authorization'] = api_key
    
    location = [latitude,longitude]
    location_str = [str(x) for x in location]
    location_join = ','.join(location_str)
    
    params_dict = {
                   "query" : "transportation",
                   "ll" : location_join,
                   "categories" : "19042,19043",
                   "radius" : "2000",
                   "limit" : "50"
    }
    
    response = requests.get(url, params_dict, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f'Error {response.status_code}')

In [145]:
Transportation_data = df.apply(
                            lambda row: get_transp_POI(row['Latitude'], row['Longitude']),
                            axis=1)

In [165]:
Transportation_data[0]['context']['geo_bounds']['circle']['center']['latitude']

43.8066863

In [167]:
fs_trans_api=[]
def trans_POI(data):
    for row in data:
        for val in row['results']:
            try:
                fs_trans_api.append(
                                [
                                   val['name'],
                                   val['popularity'],
                                   row['context']['geo_bounds']['circle']['center']['latitude'],
                                ]
                                    )
            except KeyError:
                continue

In [168]:
trans_POI(Transportation_data)

In [170]:
df_transportation=pd.DataFrame(data=fs_trans_api, columns=['name','popularity','latitude'])

In [176]:
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [177]:
df_restaurants.head()

,name,category,Postal_code,popularity,rating,price
0,Subway,Deli,M1B,0.860107,6.4,1
1,Wendy's,Fast Food Restaurant,M1B,0.995847,6.2,1
2,Harvey's,Fast Food Restaurant,M1B,0.982336,6.3,2
3,Caribbean Wave,American Restaurant,M1B,0.988098,6.4,1
4,Swiss Chalet Rotisserie & Grill,Restaurant,M1B,0.948866,5.2,2


In [178]:
df_transportation.head()

,name,popularity,latitude
0,Bus Stop: 85 & 116,0.903790,43.806686
1,"Bus Stop: 116, Cinemart",0.920266,43.806686
2,Highland Creek Overpass,0.456998,43.806686
3,Bus stop: 95 Westbound,0.318424,43.784535
4,Bus Stop: 116,0.332937,43.763573


In [179]:
def get_venue_POI(latitude,longitude):
    
    url = "https://api.foursquare.com/v3/places/search?fields=name,location,categories,rating,popularity,price"
    
    headers = {"Accept": "application/json"}
    headers['Authorization'] = api_key
    
    location = [latitude,longitude]
    location_str = [str(x) for x in location]
    location_join = ','.join(location_str)
    
    params_dict = {
                   "ll" : location_join,
                   #"categories" : 
                   "radius" : "2000",
                   "limit" : "50"
    }
    
    response = requests.get(url, params_dict, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f'Error {response.status_code}')

In [180]:
Venue_data = df.apply(
                    lambda row: get_venue_POI(row['Latitude'], row['Longitude']),
                    axis=1)

In [197]:
Venue_data[0]['results'][0]['categories'][0]

{'id': 17106,
 'name': 'Packaging Supply Store',
 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/default_',
  'suffix': '.png'}}

In [204]:
fs_venue_api=[]
def venue_POI(data):
    for row in data:
        for val in row['results']:
            try:
                fs_venue_api.append(
                                [
                                   val['name'],
                                   val['categories'][0]['name'],
                                   val['location']['postcode'],
                                   val['popularity'],
                                ]
                                    )
            except KeyError:
                continue
            except IndexError:
                continue


In [205]:
venue_POI(Venue_data)

In [206]:
df_venues = pd.DataFrame(fs_venue_api, columns = ['name','category','Postal_code','popularity'])

In [210]:
df_venues['Postal_code'] = df_venues['Postal_code'].apply(lambda x: x[:3])

In [211]:
df_venues.head()

,name,category,Postal_code,popularity
0,Rex Pak Food Packaging Ltd,Packaging Supply Store,M1B,0.891394
1,Canadian Appliance Source Whitby,Home Service,L1R,0.917554
2,The Home Depot,Home Improvement Service,M1B,0.998187
3,T Hamilton & Son Roofing Inc,Roofer,M1B,0.980982
4,Subway,Deli,M1B,0.859652
